<a href="https://colab.research.google.com/github/IANGECHUKI176/deeplearning/blob/main/pytorch/convnets/Resnets_original_paper_implemenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self,in_channels,out_channels,stride = 1):
        super(BasicBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size = 3,stride = stride ,padding = 1,bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3,stride = 1,padding = 1,bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,self.expansion * out_channels,kernel_size = 1,stride = stride,bias = False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    def forward(self,x):
        identity = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += self.shortcut(identity)
        x = F.relu(x)
        return x


In [3]:
class BottleneckBlock(nn.Module):
    expansion = 4
    def __init__(self,in_channels,out_channels,stride = 1):
        super(BottleneckBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size = 1,stride = 1,bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size = 3,stride = stride,padding = 1,bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels,out_channels * self.expansion,kernel_size = 1,stride=1,bias = False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels :
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,self.expansion*out_channels,kernel_size = 1,stride = stride,bias = False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    def forward(self,x):
        identity = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))

        identity = self.shortcut(identity)
        x += identity
        x = self.relu(x)
        return x

In [4]:
class Resnet(nn.Module):
    def __init__(self,block,layers,num_classes):
        super(Resnet,self).__init__()
        self.in_channels =64

        self.conv1 = nn.Conv2d(3,64,kernel_size = 7,stride = 2,padding = 3,bias = False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size = 3,stride = 2,padding = 1)

        self.layer1 = self._make_layer(block,64,layers[0],stride = 1)
        self.layer2 = self._make_layer(block,128,layers[1],stride = 2)
        self.layer3 = self._make_layer(block,256,layers[2],stride = 2)
        self.layer4 = self._make_layer(block,512,layers[3],stride = 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion,num_classes)
    def _make_layer(self,block,out_channels,blocks,stride =1):
        layers = []
        layers.append(block(self.in_channels,out_channels,stride))
        self.in_channels = out_channels * block.expansion
        for i in range(1,blocks):
            layers.append(block(self.in_channels,out_channels))
        return nn.Sequential(*layers)
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x= torch.flatten(x,1)
        x = self.fc(x)
        return x

In [5]:
input = torch.randn(64,3,224,224)

def resnet18(num_classes=10):
    return Resnet(BasicBlock,[2, 2, 2, 2],num_classes)

def resnet34(num_classes=10):
    return Resnet(BasicBlock,[3, 4, 6, 3],num_classes)

In [6]:
def resnet50(num_classes = 10):
    return Resnet(BottleneckBlock,[3, 4, 6, 3],num_classes)

def resnet101(num_classes = 10):
    return Resnet(BottleneckBlock,[3, 4, 23, 3],num_classes)

def resnet152(num_classes = 10):
    return Resnet(BottleneckBlock,[3, 8, 36, 3],num_classes)